# Automated ML



In [ ]:
import pandas as pd

## Azure-related
import azureml.core
print("SDK version:", azureml.core.VERSION)


##  Create compute cluster
        # Get WS details and Subscription details

In [36]:
from azureml.core import Workspace, Experiment, Run
#ws = Workspace.get(name="udacity-project")
ws = Workspace.from_config() 

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')




ModuleNotFoundError: No module named 'azureml'

In [ ]:
# choose a name for experiment
experiment_name = 'incidents-exp'
experiment=Experiment(ws, experiment_name)

run = exp.start_logging()

## Create Cluster

In [ ]:
### Specify compute targets
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "cpu-cluster"

# Verify that the cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2',
                                                           idle_seconds_before_scaledown=2400,
                                                           min_nodes=0,
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    cpu_cluster.wait_for_completion(show_output=True)



## Dataset

### Overview
* The incident management log data is retrieved from UCI Machine learnig data set. [Incident Data)[https://archive.ics.uci.edu/ml/datasets/Incident+management+process+enriched+event+log]. The data is sourced from a Servicenow platform for IT service management.  
  
* This provides details of various incidents recorded over period of time . I have taken up this project to predict ETA(expected time of accomplishment) to understand time to resolve each incident. This helps IT department to provide ETA for customers based on time taken to resolve similar issues historically. This will also help IT department to understand if any instance will go beyond expected SLA.


from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

In [ ]:
'''
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core import Dataset

datasource = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00498/incident_event_log.zip'
dataset = Dataset.Tabular.from_delimited_files(path=datasource, header=True).to_pandas_dataframe()
#dataset.columns =['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'iris_class'] 

print(dataset.head())
'''

In [ ]:
#import source.loadData as src
import source.loadData as src
import pandas as pd

datasource = "https://archive.ics.uci.edu/ml/machine-learning-databases/00498/incident_event_log.zip"
df = src.loadData(datasource)
#df.head(2)
df.head()


In [34]:
df.head(2)

,number,incident_state,active,reassignment_count,reopen_count,sys_mod_count,made_sla,caller_id,opened_by,opened_at,...,problem_id,rfc,vendor,caused_by,closed_code,resolved_by,resolved_at,closed_at,closedat,days_to_close
0,INC0000045,New,True,0,0,0,True,Caller 2403,Opened by 8,2016-02-29 01:16:00,...,?,?,?,?,code 5,Resolved by 149,29/2/2016 11:29,2016-05-03 12:00:00,2016-05-03 12:00:00,64 days 10:44:00
1,INC0000045,Resolved,True,0,0,2,True,Caller 2403,Opened by 8,2016-02-29 01:16:00,...,?,?,?,?,code 5,Resolved by 149,29/2/2016 11:29,2016-05-03 12:00:00,2016-05-03 12:00:00,64 days 10:44:00


In [29]:
'''
import dateutil.parser
df['closed_at'] = pd.to_datetime(df['closed_at'])
df['opened_at'] = pd.to_datetime(df['opened_at'])

df['days_to_close'] = (df['closed_at'] - df['opened_at'])
df['days_to_close'].dt.components.days
#df['days_to_close'] = df['days_to_close'].date()
'''

In [35]:
x, y = src.select_features_target(df)

In [ ]:
x_train, x_test, y_train, y_test = src.train_test_data(x, y)

In [ ]:
'''
#df['opened_at'] = pd.to_datetime(df['opened_at'])
#df['sys_created_at'] = pd.to_datetime(df['sys_created_at'])
#df['sys_updated_at'] = pd.to_datetime(df['sys_updated_at'])
#df['resolved_at'] = pd.to_datetime(df['resolved_at'])
#df['opened_at'] = pd.to_datetime(df['opened_at'])
#df['closed_at'] = pd.to_datetime(df['closed_at'])
'''

In [ ]:
'''
df.astype({
'opened_at':'datetime64',
'sys_created_at':'datetime64',
'sys_updated_at':'datetime64',
'sys_updated_at':'datetime64',
'resolved_at':'datetime64',
'closed_at':'datatime64'}).dtypes
'''

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [ ]:
from azureml.train.automl import AutoMLConfig

# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes"=60,
    "max_concurrent_iterations":5,
    "primary_metric"='r2_score',
    "n_cross_validations"=5,
    
    "iteration_timeout_minutes" : 10,
    "iterations" : 10,
    "primary_metric" : 'AUC_weighted',
    "verbosity" : logging.INFO,
    "preprocess": True
    }



In [ ]:
# TODO: Put your automl config here
automl_config = AutoMLConfig(task='regression',
                             allowed_models=['KNN'],
                             label_column_name=label,
                             training_data=train_data,
                            )
'''
 automated_ml_config = AutoMLConfig(task = 'classification',
                                 debug_log = 'automated_ml_errors.log',
                                 compute_target=compute_target,
                                 path = project_folder,
                                 data_script= project_folder + "/get_data.py",
                                 model_explainability=True,
                                 **automl_settings)
'''

In [ ]:
# TODO: Submit your experiment
run = experiment.submit(automl_config, show_output=True)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
# submit an experiment
from azureml.widgets import RunDetails

automl_run = exp.submit(automl_config, show_output=True)
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)




# HYPERDRIVE

1.2  Hyperdrive Configuration¶
TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.


In [ ]:
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
ps = RandomParameterSampling(
   parameter_space={
       "--C": uniform(0.1, 0.9),
       "--max_iter": choice(10, 50, 100)
    }
)

# Specify a Policy
policy = BanditPolicy(
    evaluation_interval=3,
    slack_factor=0.1
)

# Create a SKLearn estimator for use with train.py
est = SKLearn(
    source_directory="./",
    entry_script="train.py",
    compute_target=aml_compute,
)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    primary_metric_name="Accuracy",
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=48,
    max_concurrent_runs=8,
    hyperparameter_sampling=ps,
    policy=policy,
    estimator=est
)

In [ ]:
# start the HyperDrive run
hyperdrive_run = experiment.submit(hyperdrive_config)

## Monitor Hyperdrive

In [ ]:
from azureml.widgets import RunDetails

RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

In [ ]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run)
print(hyperdrive_run.get_metrics())


hyperdrive_run.wait_for_completion(show_output=True)

In [ ]:
assert(hyperdrive_run.get_status() == "Completed")

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:






best_automl_run, best_model = automl_run.get_output()
print(best_run)
print(fitted_model)


In [ ]:
# To get your evaluation metrics as a dictionary, use the following commamd:

In [ ]:

run_metrics = run.get_metrics()
run_metrics

In [ ]:
#TODO: Save the best model

In [ ]:
best_manual_accuracy = float(max(run_metrics['Accuracy'])) * 100 
print("The best accuracy we acheived with Scikit Learn:  %.2f%%" % best_manual_accuracy)

In [ ]:
from azureml.core import get_run
run_cpu_id = 'AutoML_2e02b696-bf8a-47f4-b035-64347532b27e_9'
# run_cpu_id = 'AutoML_fd9055d1-1f4b-4484-ae70-d773ca82bd67' #get from portal
best_run = get_run(experiment, run_cpu_id)

In [ ]:
model = best_run.register_model(model_name = "best_run_automl.pkl", model_path = './outputs/')


# az ml model register -n sklearn_mnist  --asset-path outputs/sklearn_mnist_model.pkl  --experiment-name myexperiment --run-id myrunid --tag area=mnist

In [ ]:
print(best_model._final_estimator)
print(best_automl_run)

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
#az ml model deploy --ic inferenceconfig.json --dc deploymentconfig.json


from azureml.core import Model
from azureml.core.webservice import AciWebservice, Webservice


model = best_run.register_model(model_name='hyperdrive_model', model_path='./outputs/model.pkl',
                       model_framework=Model.Framework.SCIKITLEARN,
                       model_framework_version='0.19.1')
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=2,
                                                enable_app_insights=True, auth_enabled=True)  
service_name = 'my-sklearn-service'
service = Model.deploy(ws, service_name, [model], deployment_config=aci_config)
service.wait_for_deployment(show_output = True)
print(service.get_logs())

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
from azureml.core.authentication import InteractiveLoginAuthentication
import requests

auth = InteractiveLoginAuthentication()
aad_token = auth.get_authentication_header()

rest_endpoint = published_pipeline.endpoint

print("You can perform HTTP POST on URL {} to trigger this pipeline".format(rest_endpoint))

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
# specify the param when running the pipeline
response = requests.post(rest_endpoint, 
                         headers=aad_token, 
                         json={"ExperimentName": "MyRestPipeline",
                               "RunSource": "SDK",
                               "DataSetDefinitionValueAssignments": {"file_ds_param": {"SavedDataSetReference": {"Id": iris_file_ds.id}},
                                                                     "tabular_ds_param": {"SavedDataSetReference": {"Id": iris_tabular_ds.id}}}
                              }
                        )

In [ ]:
try:
    response.raise_for_status()
except Exception:    
    raise Exception('Received bad response from the endpoint: {}\n'
                    'Response Code: {}\n'
                    'Headers: {}\n'
                    'Content: {}'.format(rest_endpoint, response.status_code, response.headers, response.content))

run_id = response.json().get('Id')
print('Submitted pipeline run: ', run_id)

In [ ]:
published_pipeline_run_via_rest = PipelineRun(ws.experiments["MyRestPipeline"], run_id)
RunDetails(published_pipeline_run_via_rest).show()

In [ ]:
published_pipeline_run_via_rest.wait_for_completion()

# Install and use ONNX Runtime with Python

In [ ]:
pip install onnxruntime	      # CPU build
#pip install onnxruntime-gpu   # GPU build

In [ ]:
import onnxruntime
session = onnxruntime.InferenceSession("path to model")

In [ ]:
session.get_modelmeta()
first_input_name = session.get_inputs()[0].name
first_output_name = session.get_outputs()[0].name

In [ ]:
results = session.run(["output1", "output2"], {
                      "input1": indata1, "input2": indata2})
results = session.run([], {"input1": indata1, "input2": indata2})